# THỐNG KÊ MÁY TÍNH VÀ ỨNG DỤNG (ĐTTX)

**Khoa Công nghệ Thông tin - ĐH Khoa học Tự nhiên TP. HCM ([fit@hcmus](https://www.fit.hcmus.edu.vn/))**

*Giảng viên: Vũ Quốc Hoàng (vqhoang@fit.hcmus.edu.vn)*

# BÀI 3 - XỬ LÝ MẢNG SỐ VỚI NUMPY (Phần 3)

**Nội dung**

* [Dữ liệu `Iris Data Set`](#doc_bang)
* [Broadcasting](#broadcasting)
* [Các phép toán so sánh và luận lý](#so_sanh)
* [Truy cập các phần tử của mảng bằng danh sách các chỉ số](#truy_cap)
* [Một vài thao tác đặc biệt trên mảng](#thao_tac_dac_biet)

**Tài liệu tham khảo**

* Chương 8, 9, 10, 11 [Python Data Science Handbook (Jake VanderPlas)](https://www.amazon.com/Python-Data-Science-Handbook-Essential-dp-1098121228/dp/1098121228/)

In [1]:
import numpy as np

## <a name="doc_bang"/>Dữ liệu `Iris Data Set`

Dữ liệu [`Iris Data Set`](https://archive.ics.uci.edu/ml/datasets/Iris).

Tập tin dữ liệu: [`iris.data`](https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data).

Tham khảo: [Iris flower data set](https://en.wikipedia.org/wiki/Iris_flower_data_set).

In [2]:
iris_names = ["sepal_length", "sepal_width", "petal_length", "petal_width", "class"]
iris_dtypes = ["f4", "f4", "f4", "f4", "U20"]
iris_table = np.loadtxt("iris.data", delimiter = ",", skiprows = 0, 
                        dtype={"names": iris_names, "formats": iris_dtypes}) 

In [3]:
iris_table[:5]

array([(5.1, 3.5, 1.4, 0.2, 'Iris-setosa'),
       (4.9, 3. , 1.4, 0.2, 'Iris-setosa'),
       (4.7, 3.2, 1.3, 0.2, 'Iris-setosa'),
       (4.6, 3.1, 1.5, 0.2, 'Iris-setosa'),
       (5. , 3.6, 1.4, 0.2, 'Iris-setosa')],
      dtype=[('sepal_length', '<f4'), ('sepal_width', '<f4'), ('petal_length', '<f4'), ('petal_width', '<f4'), ('class', '<U20')])

Trích riêng các features (các cột số đo) và nhãn lớp (class)

In [4]:
iris_classes = iris_table["class"]
iris_features = np.hstack([iris_table[n].reshape(-1, 1) for n in iris_names if n != "class"])

In [5]:
print(iris_classes[:5])
print(iris_features[:5])

['Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa']
[[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]]


## <a name="broadcasting"/>Broadcasting

**Broadcasting** là kỹ thuật của NumPy giúp thao tác với các mảng (`ndarray`) có số chiều (`ndim`) và/hoặc kích thước (`shape`) khác nhau.

Cùng với vector hóa, broadcasting mang lại sự hiệu quả cho NumPy.

In [6]:
# hàm tự cộng số (scalar) c vào các phần tử của vector vec
def myadd_scalar(vec, c):
    vec_copy = vec.copy()
    for i in range(vec.size):
        vec_copy[i] += c
    return vec_copy

vec = np.arange(10)
print(vec)
print(myadd_scalar(vec, 100))       # tự cộng
print(vec + np.full_like(vec, 100)) # cộng bằng cách tạo mảng tạm và vector hóa
print(vec + 100)                    # cộng bằng broadcasting của NumPy

[0 1 2 3 4 5 6 7 8 9]
[100 101 102 103 104 105 106 107 108 109]
[100 101 102 103 104 105 106 107 108 109]
[100 101 102 103 104 105 106 107 108 109]


In [7]:
vec = np.arange(100_000)
%timeit myadd_scalar(vec, 100)
%timeit (vec + np.full_like(vec, 100))
%timeit (vec + 100)

23.6 ms ± 1.54 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
63.2 µs ± 2 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
41.3 µs ± 499 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


Các **qui tắc broadcasting**

1. Nếu 2 mảng khác số chiều (`ndim`) thì mảng ít chiều hơn sẽ được thêm chiều với kích thước 1.
2. Nếu 2 mảng cùng số chiều nhưng khác kích thước thì mảng có kích thước 1 sẽ được "độn thêm" để cùng kích thước ở tất cả các chiều.
3. Nếu sau khi thực hiện các qui tắc 1, 2 mà 2 mảng vẫn không cùng `shape` thì không broadcasting được.

Tham khảo thêm: [Broadcasting](https://numpy.org/doc/stable/user/basics.broadcasting.html).

In [8]:
# Ví dụ 1
a = np.array([1, 2, 3])
b = 2
b_like_a = np.full_like(a, b)

print(a)
print(b)
print(b_like_a)
print(a + b_like_a)
print(a + b) # broadcasting

[1 2 3]
2
[2 2 2]
[3 4 5]
[3 4 5]


In [9]:
# Ví dụ 2
a = np.array([[10*i for _ in range(3)] for i in range(4)])
b = np.array([1, 2, 3])
b_like_a = np.full_like(a, b)

print(a)
print(b)
print(b_like_a)
print(a + b_like_a)
print(a + b) # broadcasting

[[ 0  0  0]
 [10 10 10]
 [20 20 20]
 [30 30 30]]
[1 2 3]
[[1 2 3]
 [1 2 3]
 [1 2 3]
 [1 2 3]]
[[ 1  2  3]
 [11 12 13]
 [21 22 23]
 [31 32 33]]
[[ 1  2  3]
 [11 12 13]
 [21 22 23]
 [31 32 33]]


In [10]:
# Ví dụ 3
a = np.array([[10*i for _ in range(3)] for i in range(4)])
b = np.array([1, 2, 3, 4])

print(a, a.shape)
print(b, b.shape)
    
try:
    print(a + b) # không broadcasting được
except Exception as e:
    print(e)

[[ 0  0  0]
 [10 10 10]
 [20 20 20]
 [30 30 30]] (4, 3)
[1 2 3 4] (4,)
operands could not be broadcast together with shapes (4,3) (4,) 


In [11]:
# Ví dụ 4
a = np.array([0, 10, 20, 30]).reshape(-1, 1)
b = np.array([1, 2, 3])

a_like_b = np.full((a.shape[0], b.shape[0]), a)
b_like_a = np.full((a.shape[0], b.shape[0]), b)

print(a)
print(b)
print(a_like_b)
print(b_like_a)
print(a_like_b + b_like_a)
print(a + b) # broadcasting

[[ 0]
 [10]
 [20]
 [30]]
[1 2 3]
[[ 0  0  0]
 [10 10 10]
 [20 20 20]
 [30 30 30]]
[[1 2 3]
 [1 2 3]
 [1 2 3]
 [1 2 3]]
[[ 1  2  3]
 [11 12 13]
 [21 22 23]
 [31 32 33]]
[[ 1  2  3]
 [11 12 13]
 [21 22 23]
 [31 32 33]]


**Bài tập**

Từ ma trận `iris_features` (ma trận đặc trưng của `Iris Data Set`)
1. Tính vector `iris_mean` chứa trung bình 4 đặc trưng.
2. Dùng broadcasting tạo ma trận `iris_features_centered` là ma trận `iris_features - iris_mean` (phép biến đổi dữ liệu này được gọi là phép canh giữa - centering).
3. Kiểm tra lại để thấy `iris_features_centered` có trung bình tất cả các cột là 0.

In [12]:
# TODO:


**Bài tập**

Từ ma trận `iris_features_centered`
1. Tính vector `iris_sd` chứa độ lệch chuẩn 4 đặc trưng.
2. Dùng broadcasting tạo ma trận `iris_features_standardized` là ma trận `iris_features_centered/iris_sd` (phép biến đổi dữ liệu này được gọi là phép tỉ lệ - scaling).
3. Kiểm tra lại để thấy `iris_features_standardized` có trung bình tất cả các cột là 0 và độ lệch chuẩn tất cả các cột là 1. 

In [13]:
# TODO:


**Bài tập**

Từ ma trận `iris_features`, tương tự Câu 1, 2
1. Biến đổi dữ liệu để tất cả các cột đều có giá trị trong khoảng [0, 1] bằng cách trừ cho min và chia cho range (range là max - min).
3. Kiểm tra lại để thấy range của các cột đều là 1.

In [14]:
# TODO:


## <a name="so_sanh"/>Các phép toán so sánh và luận lý

Các phép toán so sánh (<, ==, !=, ...) cũng được NumPy thực hiện theo từng phần tử trong mảng với ufunc.

In [15]:
x = np.arange(10)
x

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [16]:
x <= 5

array([ True,  True,  True,  True,  True,  True, False, False, False,
       False])

In [17]:
x == 5

array([False, False, False, False, False,  True, False, False, False,
       False])

In [18]:
2*x == x**2

array([ True, False,  True, False, False, False, False, False, False,
       False])

In [19]:
x = np.arange(10).reshape(2, -1)
x

array([[0, 1, 2, 3, 4],
       [5, 6, 7, 8, 9]])

In [20]:
x % 2 == 0

array([[ True, False,  True, False,  True],
       [False,  True, False,  True, False]])

Các phép toán luận lý được nạp chồng bởi các toán tử bitwise logic: & (và), | (hoặc), ~ (phủ định), ^ (XOR).

In [21]:
x = np.arange(10).reshape(2, -1)
x

array([[0, 1, 2, 3, 4],
       [5, 6, 7, 8, 9]])

In [22]:
(x % 2 == 0) & (x < 5)

array([[ True, False,  True, False,  True],
       [False, False, False, False, False]])

In [23]:
(x % 2 == 0) | (x < 5)

array([[ True,  True,  True,  True,  True],
       [False,  True, False,  True, False]])

In [24]:
(x % 2 == 0) ^ (x < 5)

array([[False,  True, False,  True, False],
       [False,  True, False,  True, False]])

In [25]:
~(x % 2 == 0)

array([[False,  True, False,  True, False],
       [ True, False,  True, False,  True]])

Các hàm `np.count_nonzero`, `np.sum`, `np.any`, `np.all` hay được dùng để kiểm tra kết quả so sánh, luận lý.

In [26]:
x = np.arange(12).reshape(3, -1)
x

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [27]:
y = x < 5
y

array([[ True,  True,  True,  True],
       [ True, False, False, False],
       [False, False, False, False]])

In [28]:
np.count_nonzero(y)

5

In [29]:
np.sum(y)

5

In [30]:
np.sum(y, axis=0)

array([2, 1, 1, 1])

In [31]:
np.sum(y, axis=1)

array([4, 1, 0])

In [32]:
np.all(y, axis=1)

array([ True, False, False])

In [33]:
np.any(y, axis=0)

array([ True,  True,  True,  True])

Ta có thể dùng mảng bool để chọn ra các phần tử thỏa mãn điều kiện nào đó (kĩ thuật này được gọi là **masking**).

In [34]:
print(x)
print(y)

[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]]
[[ True  True  True  True]
 [ True False False False]
 [False False False False]]


In [35]:
x[y]

array([0, 1, 2, 3, 4])

In [36]:
x[~y]

array([ 5,  6,  7,  8,  9, 10, 11])

In [37]:
x[(5 < x) & (x < 10)]

array([6, 7, 8, 9])

Ví dụ sau đếm số lượng hoa Iris thuộc giống setosa.

In [38]:
np.unique(iris_classes)

array(['Iris-setosa', 'Iris-versicolor', 'Iris-virginica'], dtype='<U20')

In [39]:
np.sum(iris_classes == "Iris-setosa")

50

Ví dụ sau đếm số lượng hoa Iris có sepal length lớn hơn 5.

In [40]:
np.sum(iris_features[:, 0] > 5)

118

Ví dụ sau tính trung bình sepal length của giống setosa.

In [41]:
iris_features[iris_classes == "Iris-setosa", 0].mean()

5.006

**Bài tập**

Đếm số lượng hoa Iris giống virginica có sepal width lớn hơn 3.

In [42]:
# TODO:


**Bài tập**

Cho biết min, max, mean của sepal width của các hoa Iris giống virginica.

In [43]:
# TODO:


## <a name="truy_cap"/>Truy cập các phần tử của mảng bằng danh sách các chỉ số

Ta đã thấy cách dùng chỉ số (index), trượt (slice) và mảng luận lý (mask) để truy cập các phần tử của mảng `ndarray`.

In [44]:
x = 10*np.arange(10)

print(x)
print(x[5])
print(x[:5])
print(x[x > 50])

[ 0 10 20 30 40 50 60 70 80 90]
50
[ 0 10 20 30 40]
[60 70 80 90]


Ta cũng có thể dùng mảng (hay danh sách) các chỉ số để lấy về tập các phần tử (kỹ thuật này được gọi là **fancy indexing**).

In [45]:
print(x)
print([x[3], x[5], x[7], x[9]])
print(x[[3, 5, 7, 9]])
print(x[[i for i in range(3, 10, 2)]])

[ 0 10 20 30 40 50 60 70 80 90]
[30, 50, 70, 90]
[30 50 70 90]
[30 50 70 90]


Mảng kết quả trả về có dạng như mảng index.

In [46]:
ind = np.array([
    [0, 3],
    [2, 4]
])

print(ind)
print(x[ind])

[[0 3]
 [2 4]]
[[ 0 30]
 [20 40]]


Fancy index cũng có thể dùng trên mảng nhiều chiều. Khi đó ta cần danh sách chỉ số trên từng trục.

In [47]:
x = np.arange(12).reshape(3, -1)
x

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [48]:
row_ind = [0, 0, -1]
col_ind = [1, 0, -1]
x[row_ind, col_ind]

array([ 1,  0, 11])

Ta cũng có thể dùng kết hợp tất cả các kĩ thuật trên (index, slice, mask, fancy index).

In [49]:
x = np.arange(20).reshape(4, -1)
x

array([[ 0,  1,  2,  3,  4],
       [ 5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14],
       [15, 16, 17, 18, 19]])

In [50]:
x[:, [0, -1, 2]]

array([[ 0,  4,  2],
       [ 5,  9,  7],
       [10, 14, 12],
       [15, 19, 17]])

In [51]:
x[[0, -1],:] = 0
x

array([[ 0,  0,  0,  0,  0],
       [ 5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14],
       [ 0,  0,  0,  0,  0]])

**Bài tập**

Cho mảng `a` vuông kích thước 5x5 như bên dưới
1. Dùng fancy index đặt tất cả các phần tử trên đường chéo chính thành 0.
2. Nghĩ các cách Pythonic khác nhau để thực hiện công việc 1.

In [52]:
a = np.arange(25).reshape(5, -1)
a

array([[ 0,  1,  2,  3,  4],
       [ 5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14],
       [15, 16, 17, 18, 19],
       [20, 21, 22, 23, 24]])

In [53]:
# TODO:


## <a name="thao_tac_dac_biet"/>Một vài thao tác đặc biệt trên mảng

### Sắp xếp mảng bằng hàm `np.sort` và `np.argsort`

In [54]:
x = np.random.randint(0, 100, 10)
x

array([68,  6, 54, 86, 67, 83,  8, 49, 26, 62])

In [55]:
print(np.sort(x))
print(np.argsort(x))
print(x)
print(x[np.argsort(x)])

[ 6  8 26 49 54 62 67 68 83 86]
[1 6 8 7 2 9 4 0 5 3]
[68  6 54 86 67 83  8 49 26 62]
[ 6  8 26 49 54 62 67 68 83 86]


In [56]:
x.sort()
print(x)

[ 6  8 26 49 54 62 67 68 83 86]


In [57]:
x = np.random.randint(0, 100, (4, 5))
x

array([[20, 50, 11, 31,  0],
       [69, 46, 22, 19, 89],
       [17, 87, 32, 17, 21],
       [ 6, 46, 64,  1, 38]])

In [58]:
np.sort(x)

array([[ 0, 11, 20, 31, 50],
       [19, 22, 46, 69, 89],
       [17, 17, 21, 32, 87],
       [ 1,  6, 38, 46, 64]])

In [59]:
np.sort(x, axis=0)

array([[ 6, 46, 11,  1,  0],
       [17, 46, 22, 17, 21],
       [20, 50, 32, 19, 38],
       [69, 87, 64, 31, 89]])

In [60]:
np.sort(x, axis=1)

array([[ 0, 11, 20, 31, 50],
       [19, 22, 46, 69, 89],
       [17, 17, 21, 32, 87],
       [ 1,  6, 38, 46, 64]])

### Kiểm tra các giá trị duy nhất trong mảng với `np.unique`

In [61]:
x = np.random.randint(0, 3, 10)
x

array([0, 0, 1, 0, 0, 2, 1, 1, 1, 0])

In [62]:
np.unique(x)

array([0, 1, 2])

In [63]:
_, ind = np.unique(x, return_index=True)
ind

array([0, 2, 5], dtype=int64)

In [64]:
_, counts = np.unique(x, return_counts=True)
counts

array([5, 4, 1], dtype=int64)

### Chuyển vị (transpose) ma trận (biến dòng thành cột, cột thành dòng) với thuộc tính `.T`

In [65]:
x = np.random.randint(0, 10, (3, 4))
x

array([[5, 5, 3, 8],
       [8, 3, 8, 3],
       [7, 2, 0, 2]])

In [66]:
x.T

array([[5, 8, 7],
       [5, 3, 2],
       [3, 8, 0],
       [8, 3, 2]])

In [67]:
x.reshape(4, 3)

array([[5, 5, 3],
       [8, 8, 3],
       [8, 3, 7],
       [2, 0, 2]])

### Nhân ma trận (matrix multiplication) với toán tử `@` hay hàm `np.dot`

In [68]:
a = np.arange(6).reshape(2, -1)
print(a)
print(a.T)

[[0 1 2]
 [3 4 5]]
[[0 3]
 [1 4]
 [2 5]]


In [69]:
a * a

array([[ 0,  1,  4],
       [ 9, 16, 25]])

In [70]:
a @ a.T

array([[ 5, 14],
       [14, 50]])

In [71]:
np.dot(a, a.T)

array([[ 5, 14],
       [14, 50]])

In [72]:
a.dot(a.T)

array([[ 5, 14],
       [14, 50]])

### Đảo ngược mảng với `np.flip`

In [73]:
x = np.random.randint(0, 10, (3, 4))
x

array([[6, 8, 7, 8],
       [5, 5, 6, 0],
       [8, 9, 8, 6]])

In [74]:
np.flip(x)

array([[6, 8, 9, 8],
       [0, 6, 5, 5],
       [8, 7, 8, 6]])

In [75]:
np.flip(x, axis=0)

array([[8, 9, 8, 6],
       [5, 5, 6, 0],
       [6, 8, 7, 8]])

In [76]:
np.flip(x, axis=1)

array([[8, 7, 8, 6],
       [0, 6, 5, 5],
       [6, 8, 9, 8]])

In [77]:
print(x)
x[:, -1] = np.flip(x[:, -1])
print(x)

[[6 8 7 8]
 [5 5 6 0]
 [8 9 8 6]]
[[6 8 7 6]
 [5 5 6 0]
 [8 9 8 8]]
